In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

import matplotlib.pyplot as plt

In [2]:
accel = pd.read_csv("Data/Postures Ezra/Accelerometer.csv")
gyro = pd.read_csv("Data/Postures Ezra/Gyroscope.csv")
linAccel = pd.read_csv("Data/Postures Ezra/Linear Accelerometer.csv")

In [3]:
Hz = 100
times = np.array(list(range(1, int(max(gyro["Time (s)"].to_numpy())) * Hz))) / Hz # 100Hz

interpData = pd.DataFrame()

adjGyro = pd.DataFrame()
adjAccel = pd.DataFrame()
adjLinAccel = pd.DataFrame()

interpData["Time (s)"] = times
interpData["Gyro X (rad/s)"] = np.interp(times, gyro["Time (s)"], gyro["X (rad/s)"])
interpData["Gyro Y (rad/s)"] = np.interp(times, gyro["Time (s)"], gyro["Y (rad/s)"])
interpData["Gyro Z (rad/s)"] = np.interp(times, gyro["Time (s)"], gyro["Z (rad/s)"])
interpData["Accel X (m/s^2)"] = np.interp(times, accel["Time (s)"], accel["X (m/s^2)"])
interpData["Accel Y (m/s^2)"] = np.interp(times, accel["Time (s)"], accel["Y (m/s^2)"])
interpData["Accel Z (m/s^2)"] = np.interp(times, accel["Time (s)"], accel["Z (m/s^2)"])
interpData["Lin Accel X (m/s^2)"] = np.interp(times, linAccel["Time (s)"], linAccel["X (m/s^2)"])
interpData["Lin Accel Y (m/s^2)"] = np.interp(times, linAccel["Time (s)"], linAccel["Y (m/s^2)"])
interpData["Lin Accel Z (m/s^2)"] = np.interp(times, linAccel["Time (s)"], linAccel["Z (m/s^2)"])

In [ ]:
normData = interpData
normData[normData.columns[4]] = (normData[normData.columns[4]] - normData[normData.columns[4]].max()) / (normData[normData.columns[4]].max() - normData[normData.columns[4]].min())
normData[normData.columns[5]] = (normData[normData.columns[5]] - normData[normData.columns[5]].max()) / (normData[normData.columns[5]].max() - normData[normData.columns[5]].min())
normData[normData.columns[6]] = (normData[normData.columns[6]] - normData[normData.columns[6]].max()) / (normData[normData.columns[6]].max() - normData[normData.columns[6]].min())

# Show figure to label position
fig = px.line(normData[normData.columns[4:7]])
maxTime = 42 * 60 * Hz
postureInterval = 2 * 60 * Hz
transTime = 10                           # Add 10 seconds needed to move to new position
for i in range(0, maxTime + postureInterval, postureInterval):
    fig.add_vline(x = i + transTime * Hz)
fig.show()

# Remove data after experiment is done
normData = normData.drop(range(maxTime + transTime * Hz, len(normData)))

In [ ]:
# Label order of experiment
labelOrder = ['L', 'R', 'L', 'F', 'L', 'B', 'L', 'S', 'R', 'F', 'R', 'B', 'R', 'S', 'F', 'B', 'F', 'S', 'B', 'S', 'L']

# Set start times of transitions and labels, with 10 second time for transitions
transPos = [i for i in range(0, len(normData), postureInterval)][:-1]
labelPos = [time + transTime * Hz for time in transPos]

# Set labels
labels = np.chararray(len(normData), unicode = True)
for i, times in enumerate(zip(transPos, labelPos)):
    labels[times[0]:] = 'T'
    labels[times[1]:] = labelOrder[i]

normData["Label"] = labels

# Lower size of data through change of data types
numVars = ["Gyro X (rad/s)", "Gyro Y (rad/s)", "Gyro Z (rad/s)", 
           "Accel X (m/s^2)", "Accel Y (m/s^2)", "Accel Z (m/s^2)",
           "Lin Accel X (m/s^2)", "Lin Accel Y (m/s^2)", "Lin Accel Z (m/s^2)"]
normData[numVars] = normData[numVars].astype(np.float32)

normData["Time (s)"] = (normData["Time (s)"] * 1000).astype(np.int32)
normData = normData.rename(columns = {"Time (s)": "Time (ms)"})

normData["Label"] = normData["Label"].astype('<U2')

normData.dtypes

In [ ]:
colors = np.zeros(len(normData), dtype = "<U10")
colDict = {"T": "black", "R": "green", "L": "yellow", "B": "blue", "S": "red", "F": "turquoise"}

# Function to segment the data by continuous label
def segment_data(df):
    segments = []
    current_label = df.iloc[0]['Label']
    start_idx = 0
    for idx in range(1, len(df)):
        if df.iloc[idx]['Label'] != current_label:
            segments.append(df[start_idx:idx])
            start_idx = idx
            current_label = df.iloc[idx]['Label']
    segments.append(df[start_idx:])  # add the last segment
    return segments

# Segment the data
segments = segment_data(normData)

# Create the plot
fig = go.Figure()

# New order for labels
newLabOrder = []
for label in labelOrder:
    newLabOrder.append('T')
    newLabOrder.append(label)

# Add each segment as a separate trace
legend_added = set()
for i, segment in enumerate(segments):
    label = newLabOrder[i]
    show_legend = label not in legend_added
    fig.add_trace(go.Scatter(
        x = segment['Time (ms)'],
        y = segment['Accel X (m/s^2)'],
        mode = 'lines',
        name = label,
        line = {
            "color": colDict[label]
        },
        showlegend = show_legend
    )),
    legend_added.add(label)

# Update layout
fig.update_layout(
    title = "Time Series with Different Labels",
    xaxis_title = "Time (ms)",
    yaxis_title = "Acceleration X (m/s^2)"
)

# Show the plot
fig.show()

In [3]:
# normData.to_csv("Ezra Data.csv")

normData = pd.read_csv("/Users/alexanderithakis/Documents/GitHub/ML4QS/Python3Code/_Alexander/(1) Dataset_1.csv")

In [4]:
from sklearn.neighbors import LocalOutlierFactor
X = normData.iloc[:, 1:-1]
model = LocalOutlierFactor()
model.fit(X)
y_pred = model.fit_predict(X)
X[y_pred == -1] # anomalies

,Time (ms),Gyro X (rad/s),Gyro Y (rad/s),Gyro Z (rad/s),Accel X (m/s^2),Accel Y (m/s^2),Accel Z (m/s^2),Lin Accel X (m/s^2),Lin Accel Y (m/s^2),Lin Accel Z (m/s^2)


In [ ]:
sum(y_pred == -1)

In [81]:
import numpy as np
from scipy.signal import butter, filtfilt

def lowpassFilter(data, cutoff, fs, order = 5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

# Apply filter to measurements
cutoff = 0.5
filterData = normData.copy()
filterData.iloc[:, 1:-1] = filterData.iloc[:, 1:-1].apply(lowpassFilter, args = (cutoff, Hz))

In [ ]:
# Plotting
plt.plot(normData["Time (ms)"], normData["Accel Z (m/s^2)"], label = 'Noisy signal')
plt.plot(normData["Time (ms)"], filterData["Accel Z (m/s^2)"], label = 'Filtered signal', linewidth = 2)
plt.legend()
plt.show()

In [84]:
from pykalman import KalmanFilter

# Get temporal sensor data
measures = normData.drop(columns = ["Time (ms)", "Label"])

# Define initial state
initialMeans = np.zeros(measures.shape[1])

# Define state transition matrix and observation matrix
transMatrix = np.eye(measures.shape[1])
obsMatrix = np.eye(measures.shape[1])

# Initialize the Kalman filter
kf = KalmanFilter(
    transition_matrices = transMatrix,
    observation_matrices = obsMatrix,
    initial_state_mean = initialMeans
)

# Mask invalid values
maskedMeasures = np.ma.masked_invalid(measures)

# Estimate parameters
kf = kf.em(maskedMeasures, n_iter = 5)          # iters?????????

# Apply the filter to the data
filteredMeans, filteredCov = kf.filter(maskedMeasures)

# Add filtered data to the original data table
kalmanData2 = normData.copy()
kalmanData2.iloc[:, 1:-1] = filteredMeans

In [ ]:
plt.figure(figsize = (12, 8))
plt.plot(normData['Accel Z (m/s^2)'], label = 'Original Accelerometer')
plt.plot(kalmanData['Accel Z (m/s^2)'], label='Filtered X Accelerometer')
plt.tight_layout()
plt.show()